### Итоговый проект первого года обучения: Модель для классификации следов программного обеспечения

Команда аналитиков создает сигнатуры программного обеспечения для распознавания установленного ПО на основе следов установок.
Создание каждой сигнатуры - кропотливый ручной труд, поскольку след нужно проанализировать и связать с конкретным программным продуктом.
Цель проекта - отфильтровать те следы установок, которые относятся к системному ПО (драйверы, шрифты и прочее), и исключить их из объема ручной работы.

#### Задачи:
1. Создать справочник следов по уже имеющимся результатам работ
2. Применить к нему модель машинного обучения для классификации (мы будем испольовать Наивный байесовский классификатор)
3. Оценить эффективностьь полученной модели, для принятия решения о возможности ее использования для обработки новых следов


#### Подготовка справочника

In [1]:
import pandas as pd
import numpy as np

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics

Загружаем следы установок и информацию о размеченных следах из прошлых проектов.

In [2]:
df_raw = pd.read_csv('trace_data/data_raw.csv')
df_raw

,Исходный ключ,Коннектор источника,Сигнатуры,Количество записей с неполной информацией,Статус сигнатуры,ID импорта (Сигнатура),Исходный ключ (Сигнатура),Замечания (Сигнатуры),Замечания по нормализации (Сигнатуры),Ответственное лицо за нормализацию (Сигнатуры),...,Сопоставленный след программной установки,Количество статусов инвентаризации,Статус инвентаризации,Код (Соответствие),Комментарий,Описание (Соответствие),Издатель (Соответствие),Продукт (Соответствие),Версия продукта (Соответствие),Производитель (или издатель)
0,custkey_rayventory,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,0,NaN,_AuthenticationServices_SwiftUI,framework,NaN,NaN,_AuthenticationServices_SwiftUI,1.0,NaN
1,custkey_rayventory,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,0,NaN,_AVKit_SwiftUI,framework,NaN,NaN,_AVKit_SwiftUI,1.0,NaN
2,custkey_rayventory,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,0,NaN,_CoreData_CloudKit,framework,NaN,NaN,_CoreData_CloudKit,1.0,NaN
3,custkey_rayventory,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,0,NaN,_CoreLocationUI_SwiftUI,framework,NaN,NaN,_CoreLocationUI_SwiftUI,1.0,NaN
4,custkey_rayventory,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,0,NaN,_GroupActivities_AppKit,framework,NaN,NaN,_GroupActivities_AppKit,1.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4506,custkey_rayventory,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,0,NaN,Zulu Mission Control,NaN,NaN,NaN,Zulu Mission Control,7.1.1,NaN
4507,custkey_rayventory,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,0,NaN,Закрыть все программы,NaN,NaN,NaN,Закрыть все программы,1.3,NaN
4508,custkey_rayventory,NaN,1,0,NaN,22326,asp,NaN,NaN,NaN,...,NaN,0,NaN,Майкрософт Windows 10 Pro,NaN,NaN,Microsoft Corporation,Майкрософт Windows 10 Pro,10.0.19043,Microsoft
4509,custkey_rayventory,NaN,0,0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,0,NaN,Рутокен для macOS,NaN,NaN,NaN,Рутокен для macOS,1.0,NaN


In [3]:
df_marked = pd.read_csv('trace_data/traces_marked.csv')
df_marked

,ПО/не ПО,Код (Соответствие),Комментарий,Версия продукта (Соответствие)
0,не ПО,_AppIntents_AppKit,framework,1.0
1,не ПО,_AppIntents_AppKit,framework,NaN
2,не ПО,_AppIntents_SwiftUI,framework,1.0
3,не ПО,_AppIntents_SwiftUI,framework,NaN
4,не ПО,_AppIntents_UIKit,framework,1.0
...,...,...,...,...
7636,не ПО,ZoomServices,framework,NaN
7637,непонятно,zsync,NaN,0.6.2-3ubuntu1
7638,непонятно,Zui,NaN,1.0.0
7639,непонятно,Zulu Mission Control,NaN,7.1.1


След Generic определяется 5 признаками - Код (Соответствие), Описание (Соответствие), Издатель (Соответствие), Продукт (Соответствие).

Наш справочник маркированных следов содержит только два из них - Код (Соответствие)	и Версия продукта (Соответствие).

In [4]:
#избавимся от лишних столбцов в сырых данных
df_raw = df_raw[['Код (Соответствие)', 'Описание (Соответствие)', 'Издатель (Соответствие)', 'Продукт (Соответствие)', 'Версия продукта (Соответствие)']]
df_raw.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4511 entries, 0 to 4510
Data columns (total 5 columns):
 #   Column                          Non-Null Count  Dtype 
---  ------                          --------------  ----- 
 0   Код (Соответствие)              4511 non-null   object
 1   Описание (Соответствие)         1584 non-null   object
 2   Издатель (Соответствие)         680 non-null    object
 3   Продукт (Соответствие)          4511 non-null   object
 4   Версия продукта (Соответствие)  4510 non-null   object
dtypes: object(5)
memory usage: 176.3+ KB


In [5]:
#Проверим справочник маркированных следов на дубликаты
dupl_columns = ['Код (Соответствие)', 'Версия продукта (Соответствие)']

mask = df_marked.duplicated(subset=dupl_columns)
df_duplicates = df_marked[mask]
print(f'Число найденных дубликатов: {df_duplicates.shape[0]}')

Число найденных дубликатов: 46


In [6]:
#удалим дубликаты
df_dedupped = df_marked.drop_duplicates(subset=dupl_columns)
print(f'Результирующее число записей: {df_dedupped.shape[0]}')

Результирующее число записей: 7595


In [7]:
#объединим массив следов со справочником
df = df_raw.merge(df_marked, how='left', on=['Код (Соответствие)', 'Версия продукта (Соответствие)'])
df

,Код (Соответствие),Описание (Соответствие),Издатель (Соответствие),Продукт (Соответствие),Версия продукта (Соответствие),ПО/не ПО,Комментарий
0,_AuthenticationServices_SwiftUI,NaN,NaN,_AuthenticationServices_SwiftUI,1.0,не ПО,framework
1,_AVKit_SwiftUI,NaN,NaN,_AVKit_SwiftUI,1.0,не ПО,framework
2,_CoreData_CloudKit,NaN,NaN,_CoreData_CloudKit,1.0,не ПО,framework
3,_CoreLocationUI_SwiftUI,NaN,NaN,_CoreLocationUI_SwiftUI,1.0,не ПО,framework
4,_GroupActivities_AppKit,NaN,NaN,_GroupActivities_AppKit,1.0,не ПО,framework
...,...,...,...,...,...,...,...
4600,Zulu Mission Control,NaN,NaN,Zulu Mission Control,7.1.1,непонятно,NaN
4601,Закрыть все программы,NaN,NaN,Закрыть все программы,1.3,непонятно,NaN
4602,Майкрософт Windows 10 Pro,NaN,Microsoft Corporation,Майкрософт Windows 10 Pro,10.0.19043,ПО,NaN
4603,Рутокен для macOS,NaN,NaN,Рутокен для macOS,1.0,не ПО,NaN


In [8]:
#посмотрим, как выглядят метки
df['ПО/не ПО'].value_counts()

не ПО        2027
ПО           1856
непонятно     721
Name: ПО/не ПО, dtype: int64

In [9]:
#заменим метки, чтобы получить две понятные категории
df['label'] = df['ПО/не ПО'].apply(lambda x: 'software' if x=='ПО' or x==np.nan else 'non-software')
df = df.drop(['ПО/не ПО'], axis=1)
df['label'].value_counts()

non-software    2749
software        1856
Name: label, dtype: int64

In [10]:
#переименуем нужные нам столбцы в короткие названия
df.rename(columns = {'Код (Соответствие)': 'generic_key', 'Версия продукта (Соответствие)': 'product_version',\
    'Издатель (Соответствие)': 'publisher', 'Описание (Соответствие)': 'description', \
    'Продукт (Соответствие)': 'product', 'Комментарий': 'remarks'}, inplace=True)
df

,generic_key,description,publisher,product,product_version,remarks,label
0,_AuthenticationServices_SwiftUI,NaN,NaN,_AuthenticationServices_SwiftUI,1.0,framework,non-software
1,_AVKit_SwiftUI,NaN,NaN,_AVKit_SwiftUI,1.0,framework,non-software
2,_CoreData_CloudKit,NaN,NaN,_CoreData_CloudKit,1.0,framework,non-software
3,_CoreLocationUI_SwiftUI,NaN,NaN,_CoreLocationUI_SwiftUI,1.0,framework,non-software
4,_GroupActivities_AppKit,NaN,NaN,_GroupActivities_AppKit,1.0,framework,non-software
...,...,...,...,...,...,...,...
4600,Zulu Mission Control,NaN,NaN,Zulu Mission Control,7.1.1,NaN,non-software
4601,Закрыть все программы,NaN,NaN,Закрыть все программы,1.3,NaN,non-software
4602,Майкрософт Windows 10 Pro,NaN,Microsoft Corporation,Майкрософт Windows 10 Pro,10.0.19043,NaN,software
4603,Рутокен для macOS,NaN,NaN,Рутокен для macOS,1.0,NaN,non-software


#### Преобразование данных

Наш массив готов для дальнейшего преобразования и построения модели.

Идея состоит в том, чтобы разделить описывающие след строковые признаки на ряд фрагментов. Теперь наша задача будет очень похожа на спам фильтр. Мы попробуем применить Наивный байесовский классификатор для классификации каждой строки по двум классам - software и non-software.

In [11]:
#создаем функцию для обработки поля code - мы хотим использовать ряд символов в качестве разделителей и получить из одной строки список коротких фрагментов

def process_code(symbol_string):
    separator_symbols = ['_', '.', ':', '<', '>', '-']
    ignored_symbols = []
    
    count_sep = 0
    for sep in separator_symbols:
        count_sep += separator_symbols.count(sep)
    
    dropped_list = []   #для записи пропущенных символов
    string_cleaned = ''
    
    for position, symbol in enumerate(symbol_string):
        if symbol in separator_symbols:
            string_cleaned += ' '+symbol
            dropped_list.append(symbol)
        elif symbol in ignored_symbols:
            dropped_list.append(symbol)
        else:
            if position < len(symbol_string)-1 \
                and symbol_string[position].upper() == symbol_string[position] \
                and symbol_string[position+1].lower() == symbol_string[position+1]:
                string_cleaned += ' '+symbol
            # elif position < len(symbol_string)-1 \
            #     and symbol_string[position].lower() == symbol_string[position] \
            #     and symbol_string[position+1].upper() == symbol_string[position+1]:
            #     string_cleaned += symbol+' '
            else:
                string_cleaned += symbol
                
    # if len(dropped_list) > 0:
    #     string_cleaned += ' '+ ' '.join(list(set(dropped_list)))
    
    return string_cleaned

In [12]:
#сократим признак версии продукта до 3 октетов
def process_version(symbol_string, dot_limit=3, cut_tail=0):
    string_cleaned = ''
    dots = 0
    
    if symbol_string is not np.nan:
        dots = symbol_string.count('.')
        dot_limit = min(dot_limit, dots-cut_tail)
        count_dots = 0
        string_cleaned = ''
        for symbol in symbol_string:
            if symbol == '.':
                count_dots += 1
            if count_dots == dot_limit+1 or symbol == ' ':
                break
            string_cleaned += symbol
    return string_cleaned

In [13]:
#удалим неинформативные слова в описании следа
def process_description(symbol_string):  
    ignored_tokens = ['RUS']
    
    if symbol_string is not np.nan:
        for token in ignored_tokens:
            if symbol_string.count(token) > 0:
                symbol_string = symbol_string.replace(token, '')
    
    return symbol_string

Соберем общий признак keyword из полученных фрагментов ("тело письма" в примере со спам-фильтром).

In [14]:
def merge_keywords(r, use_version=True, use_publisher=True, use_description=True):
    keywords = r.code_cleaned
    if use_version:
        if r.version_cleaned is not np.nan:
            keywords += ' '+r.version_cleaned
    if use_publisher:
        if r.publisher is not np.nan:
            keywords += ' '+r.publisher
    if use_description:
        if r.description_cleaned is not np.nan:
            keywords += ' '+r.description_cleaned
    return keywords

In [15]:
df['code_cleaned'] = df['generic_key'].apply(process_code)
df['version_cleaned'] = df['product_version'].apply(process_version)
df['description_cleaned'] = df['description'].apply(process_description)

df['keywords'] = df.apply(merge_keywords, axis=1)

df['keywords'] = df['keywords'].str.lower()
df['keywords'] = df['keywords'].str.strip()
for count in range(5):
    df['keywords'] = df['keywords'].str.replace('  ', ' ')

df.drop(['code_cleaned', 'version_cleaned', 'description_cleaned'], axis=1, inplace=True)

df

,generic_key,description,publisher,product,product_version,remarks,label,keywords
0,_AuthenticationServices_SwiftUI,NaN,NaN,_AuthenticationServices_SwiftUI,1.0,framework,non-software,_ authentication services _ swiftui 1.0
1,_AVKit_SwiftUI,NaN,NaN,_AVKit_SwiftUI,1.0,framework,non-software,_av kit _ swiftui 1.0
2,_CoreData_CloudKit,NaN,NaN,_CoreData_CloudKit,1.0,framework,non-software,_ core data _ cloud kit 1.0
3,_CoreLocationUI_SwiftUI,NaN,NaN,_CoreLocationUI_SwiftUI,1.0,framework,non-software,_ core locationu i _ swiftui 1.0
4,_GroupActivities_AppKit,NaN,NaN,_GroupActivities_AppKit,1.0,framework,non-software,_ group activities _ app kit 1.0
...,...,...,...,...,...,...,...,...
4600,Zulu Mission Control,NaN,NaN,Zulu Mission Control,7.1.1,NaN,non-software,zulu mission control 7.1.1
4601,Закрыть все программы,NaN,NaN,Закрыть все программы,1.3,NaN,non-software,закрыть все программы 1.3
4602,Майкрософт Windows 10 Pro,NaN,Microsoft Corporation,Майкрософт Windows 10 Pro,10.0.19043,NaN,software,майкрософт windows 1 0 pro 10.0.19043 microsof...
4603,Рутокен для macOS,NaN,NaN,Рутокен для macOS,1.0,NaN,non-software,рутокен для macos 1.0


В новом поле keywords теперь содержится признак, на котором мы будем тренировать классификатор. Но сначала проверим, есть ли в данных дубликаты по паре ['keywords', 'label']

In [16]:
df.shape

(4605, 8)

In [17]:
df.drop_duplicates(['keywords', 'label'], inplace = True)
df.reset_index(drop=True, inplace=True)
df.shape

(4499, 8)

#### Создание модели

In [18]:
#разделяем факторы и целевой признак
X = df['keywords']
y = df['label']

In [19]:
#разбиваем массив на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=73)

In [20]:
#преобразуем выборки для построения модели
vectorizer = CountVectorizer()
X_train_vectorized = vectorizer.fit_transform(X_train)
X_test_vectorized = vectorizer.transform(X_test)
X_vectorized = vectorizer.transform(X)

print(X_train_vectorized.shape, X_test_vectorized.shape)

(3599, 3777) (900, 3777)


In [21]:
clf = MultinomialNB()
clf.fit(X_train_vectorized, y_train)

y_train_pred = clf.predict(X_train_vectorized)
y_test_pred = clf.predict(X_test_vectorized)

#### Анализ результатов

In [22]:
print('Для обучающей выборки')
print('-'*len('Для обучающей выборки'))
print(f"Accuracy: {round(metrics.accuracy_score(y_train, y_train_pred)*100, 2)}%")
print('')
print(metrics.classification_report(y_train, y_train_pred))

print('Для тестовой выборки')
print('-'*len('Для тестовой выборки'))
print(f"Accuracy: {round(metrics.accuracy_score(y_test, y_test_pred)*100, 2)}%")
print('')
print(metrics.classification_report(y_test, y_test_pred))

Для обучающей выборки
---------------------
Accuracy: 96.11%

              precision    recall  f1-score   support

non-software       0.97      0.96      0.97      2172
    software       0.95      0.96      0.95      1427

    accuracy                           0.96      3599
   macro avg       0.96      0.96      0.96      3599
weighted avg       0.96      0.96      0.96      3599

Для тестовой выборки
--------------------
Accuracy: 91.0%

              precision    recall  f1-score   support

non-software       0.91      0.95      0.93       543
    software       0.91      0.85      0.88       357

    accuracy                           0.91       900
   macro avg       0.91      0.90      0.90       900
weighted avg       0.91      0.91      0.91       900



<b>Вывод</b>. Наш подход показывает, что на тестовой выборке модель показывает более низкие результаты и склонна к переобучению. Основной для нас является метрика recall для класса software, так как мы хотим ошибок неверной классификации, когда модель срабатывает ложно и отбрасывает след, подлежащий анализу.

Для улучшения этой модели можно модификацировать функции преобразования, чтобы лучше выделять более информативные фрагменты.

Также дальнейшее увеличение объема выборки может улучшить способности модели.